In [34]:
%pip install --upgrade pip
%pip install install python-chess typing stockfish

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26305 sha256=580b4b096686f9c88a74686b375c26d529a9895a29cee01f85409f57e3c8fb3a
  Stored in directory: /Users/florian/Library/Caches/pip/wheels/9d/67/2f/53e3ef32ec48d11d7d60245255e2d71e908201d20c880c08ee
Successfully built typing
Note: you may need to restart the kernel to use updated packages.


In [52]:
import random
import math
import chess
from typing import Optional

class Node:
    def __init__(self, board: chess.Board, move: Optional[chess.Move], parent: Optional['Node']):
        self.children: list['Node'] = []
        self.value: int = 0
        self.visits: int = 0
        self.board = board
        self.move = move
        self.parent = parent
        
    def ubc1(self):
        if self.visits == 0:
            return float('inf')
        
        return self.value / self.visits + math.sqrt(2 * math.log(self.parent.visits) / self.visits)
    
    def best_child(self) -> 'Node':
        bestUBC1 = -float('inf')
        for child in self.children:
            ubc1Value = child.ubc1()
            if ubc1Value > bestUBC1 and not child.board.is_game_over():
                bestChild = child
                bestUBC1 = ubc1Value
        return bestChild

    def select(self) -> 'Node':
        if len(self.children) == 0:
            return self
        
        return self.best_child().select()

    def expand(self) -> 'Node':
        for move in self.board.legal_moves:
            board = self.board.copy()
            board.push(move)
            self.children.append(Node(board, move, self))
        
        return random.choice(self.children)
    
    def rollout_and_propagate(self):
        board = self.board.copy()
        while not board.is_game_over():
            board.push(random.choice(list(board.legal_moves)))

        result = 1 if board.outcome().winner == chess.WHITE else (0 if board.outcome().winner == None else -1)

        current = self
        self.visits = -1
        while current != None:
            current.visits += 1
            current.value += result
            current = current.parent

    def print_tree(self, depth: int = 0):
        print(f'{ " " * (depth * 2) }{ self.move } { self.value } / { self.visits }')
        for child in self.children:
            child.print_tree(depth + 1)


root = Node(chess.Board('4Bn1k/P3K3/5r2/1n4p1/3P4/1Q6/2p4p/3bb1NN w - - 0 1'), None, None)
root.expand()

for i in range(10000):
    root.select().expand().rollout_and_propagate()

root.best_child().move

0
-1
0
0
-1
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
-1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
1
0
0
-1
0
0
-1
-1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
-1
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
-1
-1
0
0
0
0
0
0
-1
0
-1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
-1
1
0
0
0
0
0
1
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
-1
0
0
0
0
1
0
0
0
1
0
0
0
1
0
0
0
-1
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
-1
0
-1
0
0
-1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
1
-1
0
0
0
0
0
0
0
1
0
1
0
1
0
0
0
0
-1
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
-1
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
1
0
0
-1
0
-1
-1
1
0
0
0
0
0
0

KeyboardInterrupt: 

In [ ]:
root.print_tree()

In [45]:
from stockfish import Stockfish
stockfish = Stockfish(path='stockfish')
stockfish.set_fen_position('4Bn1k/P3K3/5r2/1n4p1/3P4/1Q6/2p4p/3bb1NN w - - 0 1')
stockfish.set_elo_rating(500)
stockfish.get_best_move()

'b3h3'